In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

![Original-ResNet-18-Architecture.png](Original-ResNet-18-Architecture.png)

In [2]:
orig_model = resnet18(pretrained=True)
orig_model.eval()

/home/d.cherniuk/.conda/mark23/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/d.cherniuk/.conda/mark23/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
orig_model.layer2[0].downsample

Sequential(
  (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [10]:
# REDUCTION_RATES = [1, 2, 4, 5, 6, 7, 8]
REDUCTION_RATES = [1, 1.5, 2]
rank_map = {}
dummy_input = torch.rand(1, 3, 224, 224)

# original macs = C_i * W_k * H_k * C_o * W_o * H_o
# reduced macs = rank * C_i * W_i * H_i + rank * W_k * H_k * W_o * H_o + rank * C_o * W_o * H_o
layer = orig_model.conv1
weight = orig_model.conv1.weight.detach()
bias = orig_model.conv1.bias
with torch.no_grad(): y = layer(dummy_input)
print('Layer: conv1')
print('bias:', bias is not None)
print('Original Shape:', weight.shape)
weight = weight.reshape((weight.shape[0], weight.shape[1], -1))
print('Flattened Shape:', weight.shape)
print('Input shape:', dummy_input.shape)
print()

print('Parameters:', weight.numel())
orig_macs = torch.tensor(weight.shape).prod() * y.shape[-1] * y.shape[-2]
print('Original MACs:', orig_macs.item())
# print('Feasible ranks <', X.numel()/sum(list(X.shape))/REDUCTION_RATE)
ranks = list(int(weight.numel()/sum(list(weight.shape))/rate) for rate in REDUCTION_RATES)
print('Reduction rates:', REDUCTION_RATES)
print('Reduction ranks:', ranks)
print()
rank_map['conv1'] = ranks
y = orig_model.maxpool(y)

for layer_name in ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 
                   'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.downsample', 'layer2.1.conv1', 'layer2.1.conv2',
                   'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.downsample', 'layer3.1.conv1', 'layer3.1.conv2',
                   'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample', 'layer4.1.conv1', 'layer4.1.conv2']:
    lname, lidx, ltype = layer_name.split('.')
    lidx = int(lidx)
    layer = orig_model.__getattr__(lname)[lidx].__getattr__(ltype)
    if ltype == 'downsample': layer = layer[0]
    weight = layer.weight.detach()
    bias = orig_model.conv1.bias
    print('Layer:', layer_name)
    print('bias:', bias is not None)
    print('Original Shape:', weight.shape)
    weight = weight.reshape((weight.shape[0], weight.shape[1], -1))
    if ltype == 'downsample': weight = weight.reshape((weight.shape[0], weight.shape[1]))
    print('Flattened Shape:', weight.shape)
    
    c1 = layer.in_channels * y.shape[-1] * y.shape[-2]
    if layer_name.endswith('.0.conv1'): 
        print('Input shape:', y.shape)
        with torch.no_grad(): y = layer(y)
        print('Output shape:', y.shape)
    elif ltype == 'downsample':
        print('Input shape:', res.shape)
        with torch.no_grad(): print('Output shape:', layer(res).shape)
    else:
        print('Input shape:', y.shape)
        print('Output shape:', y.shape)
    c2 = layer.kernel_size[0] * layer.kernel_size[1] * y.shape[-1] * y.shape[-2]
    c3 = layer.out_channels * y.shape[-1] * y.shape[-2]
    if layer_name.endswith('.1.conv2'):
        res = y
    
    print('Parameters:', weight.numel())
    orig_macs = torch.tensor(weight.shape).prod() * y.shape[-1] * y.shape[-2]
    print('Original MACs:', orig_macs.item())
    ranks = list(int(weight.numel()/sum(list(weight.shape))/rate) for rate in REDUCTION_RATES)
    print('Reduction rates:', REDUCTION_RATES)
    print('Reduction ranks:', ranks)
    if ltype != 'downsample': 
        macs = [(rank * (c1 + c2 + c3) * 100/orig_macs).item() for rank in ranks]
        print('Reduced MACs(%):', [round(mac, 2) for mac in macs])
    print()
    rank_map[layer_name] = ranks
    

# X = orig_model.fc.weight.detach()
# bias = orig_model.conv1.bias
# print('Layer: fc')
# print('Original Shape:', X.shape)
# print('Parameters:', X.numel())
# print('bias:', bias is not None)
# # print('Feasible rank < ', X.numel()/sum(list(X.shape))/rate)
# ranks = list(int(X.numel()/sum(list(X.shape))/rate) for rate in REDUCTION_RATES)
# print('Feasible ranks = ', ranks)
# rank_map['fc'] = ranks

Layer: conv1
bias: False
Original Shape: torch.Size([64, 3, 7, 7])
Flattened Shape: torch.Size([64, 3, 49])
Input shape: torch.Size([1, 3, 224, 224])

Parameters: 9408
Original MACs: 118013952
Reduction rates: [1, 1.5, 2]
Reduction ranks: [81, 54, 40]

Layer: layer1.0.conv1
bias: False
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Input shape: torch.Size([1, 64, 56, 56])
Output shape: torch.Size([1, 64, 56, 56])
Parameters: 36864
Original MACs: 115605504
Reduction rates: [1, 1.5, 2]
Reduction ranks: [269, 179, 134]
Reduced MACs(%): [99.97, 66.52, 49.8]

Layer: layer1.0.conv2
bias: False
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Input shape: torch.Size([1, 64, 56, 56])
Output shape: torch.Size([1, 64, 56, 56])
Parameters: 36864
Original MACs: 115605504
Reduction rates: [1, 1.5, 2]
Reduction ranks: [269, 179, 134]
Reduced MACs(%): [99.97, 66.52, 49.8]

Layer: layer1.1.conv1
bias: False
Original Shape: torch.S

In [11]:
rank_map

{'conv1': [81, 54, 40],
 'layer1.0.conv1': [269, 179, 134],
 'layer1.0.conv2': [269, 179, 134],
 'layer1.1.conv1': [269, 179, 134],
 'layer1.1.conv2': [269, 179, 134],
 'layer2.0.conv1': [366, 244, 183],
 'layer2.0.conv2': [556, 370, 278],
 'layer2.0.downsample': [42, 28, 21],
 'layer2.1.conv1': [556, 370, 278],
 'layer2.1.conv2': [556, 370, 278],
 'layer3.0.conv1': [750, 500, 375],
 'layer3.0.conv2': [1132, 754, 566],
 'layer3.0.downsample': [85, 56, 42],
 'layer3.1.conv1': [1132, 754, 566],
 'layer3.1.conv2': [1132, 754, 566],
 'layer4.0.conv1': [1518, 1012, 759],
 'layer4.0.conv2': [2283, 1522, 1141],
 'layer4.0.downsample': [170, 113, 85],
 'layer4.1.conv1': [2283, 1522, 1141],
 'layer4.1.conv2': [2283, 1522, 1141]}

In [12]:
{k:v[2] for (k,v) in rank_map.items()}

{'conv1': 40,
 'layer1.0.conv1': 134,
 'layer1.0.conv2': 134,
 'layer1.1.conv1': 134,
 'layer1.1.conv2': 134,
 'layer2.0.conv1': 183,
 'layer2.0.conv2': 278,
 'layer2.0.downsample': 21,
 'layer2.1.conv1': 278,
 'layer2.1.conv2': 278,
 'layer3.0.conv1': 375,
 'layer3.0.conv2': 566,
 'layer3.0.downsample': 42,
 'layer3.1.conv1': 566,
 'layer3.1.conv2': 566,
 'layer4.0.conv1': 759,
 'layer4.0.conv2': 1141,
 'layer4.0.downsample': 85,
 'layer4.1.conv1': 1141,
 'layer4.1.conv2': 1141}